我需要的信息：entryID\Taxon ID of the species\sequence   OBO中的几个term: Molecular Function (MF), Biological Process (BP), and Cellular Component (CC)
https://github.com/claradepaolis/InformationAccretion (IA文件评分)

In [14]:
import numpy as np
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/home/angel/Temp/Protein/cafa-5-protein-function-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../cafa-5-protein-function-prediction/sample_submission.tsv
../cafa-5-protein-function-prediction/IA.txt
../cafa-5-protein-function-prediction/Test (Targets)/testsuperset-taxon-list.tsv
../cafa-5-protein-function-prediction/Test (Targets)/testsuperset.fasta
../cafa-5-protein-function-prediction/Train/train_taxonomy.tsv
../cafa-5-protein-function-prediction/Train/go-basic.obo
../cafa-5-protein-function-prediction/Train/train_terms.tsv
../cafa-5-protein-function-prediction/Train/train_sequences.fasta


In [1]:
!pip install Bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 518.9 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 5.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 kB 8.9 MB/s eta 0:00:00


In [3]:
#处理/Train/train_sequences.fasta
from Bio import SeqIO

In [21]:
sequences_dict = {}
for seq_record in SeqIO.parse('../cafa-5-protein-function-prediction/Train/train_sequences.fasta', "fasta"):
    sequences_dict[seq_record.id] = str(seq_record.seq)
sequences = pd.DataFrame(list(sequences_dict.items()), columns=['EntryID', 'Sequence'])

In [22]:
#处理/Train/train_annotations.csv
sequences_annotations = pd.read_csv('../cafa-5-protein-function-prediction/Train/train_taxonomy.tsv',sep='\t')

In [23]:
#把两个表合并
train = pd.merge(sequences, sequences_annotations, on='EntryID')

In [25]:
#使用obonet处理/Train/go-basic.obo
!pip install obonet

In [27]:
%%time
import networkx
import obonet
graph = obonet.read_obo('../cafa-5-protein-function-prediction/Train/go-basic.obo')

CPU times: user 5.37 s, sys: 350 ms, total: 5.72 s
Wall time: 5.72 s


In [34]:
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
#我想要所有节点的biological_process名称，整理成dataframe形式
go_terms = pd.DataFrame(list(id_to_name.items()), columns=['GO_ID', 'GO_Term'])

In [33]:
sorted(id_to_name[superterm] for superterm in networkx.descendants(graph,'GO:0000001'))
sorted(id_to_name[subterm] for subterm in networkx.ancestors(graph, 'GO:0042552'))

['central nervous system myelin formation',
 'central nervous system myelin maintenance',
 'central nervous system myelination',
 'myelin assembly',
 'myelin maintenance',
 'myelination in peripheral nervous system',
 'myelination of anterior lateral line nerve axons',
 'myelination of lateral line nerve axons',
 'myelination of posterior lateral line nerve axons',
 'negative regulation of myelination',
 'paranodal junction assembly',
 'peripheral nervous system myelin formation',
 'peripheral nervous system myelin maintenance',
 'positive regulation of myelination',
 'regulation of myelination']